In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.io import wavfile
import pandas as pd
from joblib import Parallel, delayed

In [ ]:
frequency_sampling, audio_signal  = wavfile.read("../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000/0047cd812e3db0cf28588f86b5c218822b556e6f9bc8c986919296da4b11e60080938fa69151bcb119b287b51db5e8bf6f6a82087f629cf03cfe86efab2c37af.wav")

In [ ]:
print('\nSignal shape:', audio_signal.shape)
print('Signal Datatype:', audio_signal.dtype)
print('Signal duration:', round(audio_signal.shape[0] / 
float(frequency_sampling), 2), 'seconds')

In [ ]:
audio_signal = audio_signal / np.power(2, 15)

In [ ]:
audio_signal = audio_signal [:1000]
time_axis = 1000 * np.arange(0, len(audio_signal), 1) / float(frequency_sampling)

In [ ]:
plt.plot(time_axis, audio_signal, color='blue')
plt.xlabel('Time (milliseconds)')
plt.ylabel('Amplitude')
plt.title('Input audio signal')
plt.show()

# Characterizing the audio signal
Converting the time domain signal into frequency domain. This is important because it gives a lot of information about the signal. A mathematical tool like fourier transformation can be used for this.

In [ ]:
length_signal = len(audio_signal)
half_length = np.ceil((length_signal + 1) / 2.0).astype(np.int)

In [ ]:
signal_frequency = np.fft.fft(audio_signal)

In [ ]:
signal_frequency = abs(signal_frequency[0:half_length]) / length_signal
signal_frequency **= 2

In [ ]:
len_fts = len(signal_frequency)

In [ ]:
if length_signal % 2:
   signal_frequency[1:len_fts] *= 2
else:
   signal_frequency[1:len_fts-1] *= 2

In [ ]:
signal_power = 10 * np.log10(signal_frequency)

In [ ]:
x_axis = np.arange(0, half_length, 1) * (frequency_sampling / length_signal) / 1000.0

In [ ]:
plt.figure()
plt.plot(x_axis, signal_power, color='red')
plt.xlabel('Frequency (kHz)')
plt.ylabel('Signal power (dB)')
plt.show()

# Installing libraries

In [ ]:
!pip install SpeechRecognition

In [ ]:
import speech_recognition as sr
sr.__version__

# Recognizer class

In [ ]:
r = sr.Recognizer()

# Loading an audio file

In [ ]:
bangla1 = sr.AudioFile('../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000/0047cd812e3db0cf28588f86b5c218822b556e6f9bc8c986919296da4b11e60080938fa69151bcb119b287b51db5e8bf6f6a82087f629cf03cfe86efab2c37af.wav')

In [ ]:
with bangla1 as source:
    audio = r.record(source)

In [ ]:
type(audio)

In [ ]:
#using google web speech API
r.recognize_google(audio)

In [ ]:
import os
from pydub import AudioSegment

path = "../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000/"

#Change working directory
# os.chdir(path)

audio_files = os.listdir(path )

In [ ]:
bangla2 = sr.AudioFile('../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000/005cea38ff6e71a7d1047aba774790ad04336b73fed6c02ebd144060725ada6861acf7821c9abfa408067dd345078c2eb74e34535e690957ade04ea985a13bff.wav')

In [ ]:
with bangla2 as source:
    audio2 = r.record(source)

In [ ]:
type(audio2)

In [ ]:
r.recognize_google(audio2,language = "FR")

In [ ]:
from tqdm import tqdm
import os

In [ ]:
root="../input/automatic-speech-recognition-in-wolof/audio_wav_16000/tmp/WOLOF_ASR_dataset/audio_wav_16000"

In [ ]:
def save_fn(filename):
    
    path = f"{root}/{filename}.wav"
    if os.path.exists(path):
        try:
            wolof = sr.AudioFile(path)
            with wolof as source:
                audio2 = r.record(source)
            transcript = r.recognize_google(audio2,language = "fr")
        
            return transcript
        except:
            print(path)

In [ ]:
wolof_trans

In [ ]:


Test = pd.read_csv("../input/automatic-speech-recognition-in-wolof/Test.csv")
test_list = list(Test.ID.values)
wolof_trans = Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(save_fn)(filename) for filename in tqdm(test_list)
)  
   

In [ ]:

Sub = pd.read_csv("../input/automatic-speech-recognition-in-wolof/SampleSubmission.csv")
Sub["transcription"] = wolof_trans
Sub.head()

In [ ]:
Sub.to_csv("submission.csv",index=False)